In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
import pandas as pd
import torch.nn.functional as F
import xgboost as xgb
from statsmodels.tsa.seasonal import seasonal_decompose
import optuna
from prophet import Prophet

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
import pandas as pd
import torch.nn.functional as F
import warnings
warnings.filterwarnings('ignore')

# =========================================
# Model LSTM đã được sửa lỗi và tối ưu
# =========================================
class AdvancedLSTMModel(nn.Module):
    def __init__(self, input_size=1, hidden_sizes=[64, 32], dropout=0.2, output_size=1):
        super().__init__()
        self.hidden_sizes = hidden_sizes
        self.num_layers = len(hidden_sizes)
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
        # Stacked LSTM layers
        self.lstm_layers = nn.ModuleList()
        for i in range(self.num_layers):
            input_sz = input_size if i == 0 else hidden_sizes[i-1]
            self.lstm_layers.append(nn.LSTM(input_sz, hidden_sizes[i], 
                                          batch_first=True, dropout=dropout if i < self.num_layers-1 else 0))
        
        # Batch normalization
        self.bn = nn.BatchNorm1d(hidden_sizes[-1])
        
        # Output layers với dropout
        self.dropout = nn.Dropout(dropout)
        self.fc1 = nn.Linear(hidden_sizes[-1], 32)
        self.fc2 = nn.Linear(32, output_size)
    
    def forward(self, x, hidden=None):
        batch_size = x.size(0)
        
        # Initialize hidden states nếu cần
        if hidden is None:
            hidden = self.init_hidden(batch_size)
        
        # Qua các LSTM layers - SỬA LỖI: chỉ lấy output cuối của layer cuối
        lstm_out = x
        for i, lstm in enumerate(self.lstm_layers):
            lstm_out, hidden[i] = lstm(lstm_out, hidden[i])
            if i == self.num_layers - 1:  # Chỉ layer cuối mới lấy output cuối
                x = lstm_out[:, -1, :]  # (batch, hidden_size)
            else:
                x = lstm_out  # Giữ nguyên sequence cho layer tiếp theo
        
        # Batch norm và fully connected
        x = self.bn(x)
        x = F.relu(self.fc1(self.dropout(x)))
        x = self.fc2(x)
        
        return x, hidden
    
    def init_hidden(self, batch_size):
        hidden = []
        for size in self.hidden_sizes:
            h0 = torch.zeros(1, batch_size, size, device=self.device)
            c0 = torch.zeros(1, batch_size, size, device=self.device)
            hidden.append((h0, c0))
        return hidden

# =========================================
# Data preprocessing với multivariate support
# =========================================
def load_and_split_data(file_path='data.csv', seq_length=3, test_size=0.1, val_size=0.1):
    df = pd.read_csv(file_path)
    
    # Xử lý multivariate data
    if 'Registrations' in df.columns:
        target_col = 'Registrations'
        # Tìm tất cả cột điểm (Diem_)
        feature_cols = [target_col]
        diem_cols = [col for col in df.columns if col.startswith('Diem_')]
        feature_cols.extend(diem_cols)
        
        print(f"Using features: {feature_cols}")
        
        # Điền missing values
        df[feature_cols] = df[feature_cols].fillna(method='ffill').fillna(method='bfill')
        
        # Scale features
        scaler = MinMaxScaler(feature_range=(0, 1))
        features_scaled = scaler.fit_transform(df[feature_cols].values)
        
        # Target index
        target_idx = feature_cols.index(target_col)
        
    else:
        # Fallback to original single feature
        print("Using single feature 'Registrations'")
        scaler = MinMaxScaler(feature_range=(0, 1))
        features_scaled = scaler.fit_transform(df['Registrations'].values.reshape(-1, 1))
        target_idx = 0
    
    years = df['Year'].values
    
    # Create sequences
    X, y = [], []
    for i in range(len(features_scaled) - seq_length):
        X.append(features_scaled[i:i+seq_length])
        y.append(features_scaled[i+seq_length, target_idx])  # Predict target
    
    X = np.array(X)
    y = np.array(y)
    
    # Temporal split (không shuffle)
    n_samples = len(X)
    test_start = int(n_samples * (1 - test_size))
    val_start = int(test_start * (1 - val_size))
    
    X_train = X[:val_start]
    X_val = X[val_start:test_start]
    X_test = X[test_start:]
    
    y_train = y[:val_start]
    y_val = y[val_start:test_start]
    y_test = y[test_start:]
    
    print(f"Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")
    print(f"Input features: {X.shape[2]}")
    
    return (torch.FloatTensor(X_train), torch.FloatTensor(y_train),
            torch.FloatTensor(X_val), torch.FloatTensor(y_val),
            torch.FloatTensor(X_test), torch.FloatTensor(y_test),
            scaler, years, features_scaled, target_idx)

# =========================================
# EarlyStopping đã hoàn thiện
# =========================================
class EarlyStopping:
    def __init__(self, patience=15, min_delta=1e-4, restore_best_weights=True):
        self.patience = patience
        self.min_delta = min_delta
        self.restore_best_weights = restore_best_weights
        self.best_loss = None
        self.counter = 0
        self.best_weights = None
        self.best_epoch = 0
    
    def __call__(self, val_loss, model, epoch):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.save_checkpoint(model)
            self.best_epoch = epoch
        elif val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
            self.save_checkpoint(model)
            self.best_epoch = epoch
        else:
            self.counter += 1
        
        if self.counter >= self.patience:
            if self.restore_best_weights:
                model.load_state_dict(self.best_weights)
                print(f"Restored best weights from epoch {self.best_epoch}")
            return True
        return False
    
    def save_checkpoint(self, model):
        self.best_weights = model.state_dict().copy()

# =========================================
# Training function hoàn chỉnh
# =========================================
def train_model(X_train, y_train, X_val, y_val, input_size, epochs=300, batch_size=16, lr=0.001):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Training on {device}")
    
    model = AdvancedLSTMModel(input_size=input_size, hidden_sizes=[64, 32], dropout=0.2).to(device)
    
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=10, factor=0.5, verbose=True)
    
    early_stopping = EarlyStopping(patience=20, min_delta=1e-4)
    
    train_losses, val_losses = [], []
    
    for epoch in range(epochs):
        # Training phase
        model.train()
        train_loss = 0
        num_batches = 0
        
        # Mini-batch training
        for i in range(0, len(X_train), batch_size):
            batch_X = X_train[i:i+batch_size].to(device)
            batch_y = y_train[i:i+batch_size].to(device)
            
            optimizer.zero_grad()
            outputs, _ = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            
            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            train_loss += loss.item()
            num_batches += 1
        
        avg_train_loss = train_loss / num_batches
        train_losses.append(avg_train_loss)
        
        # Validation phase
        model.eval()
        with torch.no_grad():
            val_outputs, _ = model(X_val.to(device))
            val_loss = criterion(val_outputs, y_val.to(device)).item()
            val_losses.append(val_loss)
        
        # Learning rate scheduling
        scheduler.step(val_loss)
        
        # Logging
        if epoch % 20 == 0 or epoch < 10:
            current_lr = optimizer.param_groups[0]['lr']
            print(f'Epoch {epoch:3d}: Train={avg_train_loss:.6f}, Val={val_loss:.6f}, LR={current_lr:.2e}')
        
        # Early stopping
        if early_stopping(val_loss, model, epoch):
            print(f"Early stopping at epoch {epoch}")
            break
    
    # Save final best model
    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'best_val_loss': early_stopping.best_loss,
        'train_losses': train_losses,
        'val_losses': val_losses
    }, 'advanced_lstm_model.pth')
    
    print(f"Best validation loss: {early_stopping.best_loss:.6f}")
    print("Model checkpoint saved to 'advanced_lstm_model.pth'")
    
    return model, train_losses, val_losses

# =========================================
# Evaluation function cải tiến
# =========================================
def evaluate_model(model, X_test, y_test, scaler, feature_cols, target_idx):
    device = next(model.parameters()).device
    model.eval()
    
    with torch.no_grad():
        test_pred = model(X_test.to(device))[0].cpu().numpy()
        y_test_np = y_test.numpy()
        
        # Inverse transform cho multivariate
        if len(feature_cols) > 1:
            # Tạo dummy features cho inverse transform
            dummy_features = np.zeros((len(test_pred), len(feature_cols)))
            dummy_features[:, target_idx] = test_pred.flatten()
            test_pred_orig = scaler.inverse_transform(dummy_features)[:, target_idx]
            
            dummy_true = np.zeros((len(y_test_np), len(feature_cols)))
            dummy_true[:, target_idx] = y_test_np
            y_test_orig = scaler.inverse_transform(dummy_true)[:, target_idx]
        else:
            test_pred_orig = scaler.inverse_transform(test_pred)
            y_test_orig = scaler.inverse_transform(y_test_np.reshape(-1, 1)).flatten()
        
        # Metrics
        mse = mean_squared_error(y_test_orig, test_pred_orig)
        mae = mean_absolute_error(y_test_orig, test_pred_orig)
        mape = np.mean(np.abs((y_test_orig - test_pred_orig) / (y_test_orig + 1e-8))) * 100
        
        rmse = np.sqrt(mse)
        
        print(f"Test Results:")
        print(f"  MSE: {mse:,.2f}")
        print(f"  RMSE: {rmse:,.2f}")
        print(f"  MAE: {mae:,.2f}")
        print(f"  MAPE: {mape:.2f}%")
        
        # R² score
        ss_res = np.sum((y_test_orig - test_pred_orig) ** 2)
        ss_tot = np.sum((y_test_orig - np.mean(y_test_orig)) ** 2)
        r2 = 1 - (ss_res / ss_tot)
        print(f"  R²: {r2:.4f}")
    
    return test_pred_orig, y_test_orig

# =========================================
# Multi-step prediction cải tiến
# =========================================
def predict_future(model, scaler, last_sequence, future_years=[2025, 2026, 2027], 
                  feature_cols=None, target_idx=0, num_features=1):
    model.eval()
    device = next(model.parameters()).device
    
    predictions = []
    current_seq = last_sequence.copy().astype(np.float32)
    
    with torch.no_grad():
        for i, year in enumerate(future_years):
            input_seq = torch.FloatTensor(current_seq).unsqueeze(0).to(device)
            pred, _ = model(input_seq)
            pred_val = pred.cpu().numpy()[0, 0]
            predictions.append(pred_val)
            
            # Update sequence cho multi-step
            if num_features > 1:
                # Pad dummy values cho other features (có thể cải tiến bằng trend extrapolation)
                new_row = np.zeros(num_features)
                new_row[target_idx] = pred_val
                # Copy last values của other features
                for j in range(num_features):
                    if j != target_idx:
                        new_row[j] = current_seq[-1, j]
                current_seq = np.vstack([current_seq[1:], new_row])
            else:
                current_seq = np.append(current_seq[1:], pred_val).reshape(-1, 1)
    
    # Inverse scale
    if num_features > 1:
        dummy_features = np.zeros((len(predictions), num_features))
        dummy_features[:, target_idx] = predictions
        predictions_orig = scaler.inverse_transform(dummy_features)[:, target_idx]
    else:
        predictions_orig = scaler.inverse_transform(np.array(predictions).reshape(-1, 1)).flatten()
    
    return predictions_orig

# =========================================
# Plotting function cải tiến
# =========================================
def plot_results(train_losses, val_losses, years, registrations, future_years, predictions, 
                test_pred=None, test_true=None, save_fig=False):
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
    
    # Training curves
    ax1.plot(train_losses, label='Train Loss', alpha=0.7)
    ax1.plot(val_losses, label='Val Loss', alpha=0.7)
    ax1.set_title('Training & Validation Loss')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Main prediction plot
    ax2.plot(years, registrations, 'o-', label='Historical Data', linewidth=2, markersize=6)
    if test_pred is not None and test_true is not None:
        test_years = years[-len(test_pred):]
        ax2.plot(test_years, test_true, 's', label='Test True', markersize=8, alpha=0.8)
        ax2.plot(test_years, test_pred, 'x', label='Test Prediction', markersize=8, alpha=0.8)
    ax2.plot(future_years, predictions, 'o--', label='Future Prediction', 
             linewidth=3, markersize=10, color='red')
    ax2.set_title('Registration Trends Prediction')
    ax2.set_xlabel('Year')
    ax2.set_ylabel('Registrations')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # Residuals plot
    if test_pred is not None and test_true is not None:
        residuals = test_true - test_pred
        ax3.scatter(test_true, residuals, alpha=0.6)
        ax3.axhline(y=0, color='r', linestyle='--')
        ax3.set_title('Residuals Plot')
        ax3.set_xlabel('True Values')
        ax3.set_ylabel('Residuals')
        ax3.grid(True, alpha=0.3)
    
    # Prediction vs Actual
    if test_pred is not None and test_true is not None:
        ax4.scatter(test_true, test_pred, alpha=0.7)
        min_val = min(test_true.min(), test_pred.min())
        max_val = max(test_true.max(), test_pred.max())
        ax4.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2)
        ax4.set_title('Predicted vs Actual')
        ax4.set_xlabel('Actual')
        ax4.set_ylabel('Predicted')
        ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    if save_fig:
        plt.savefig('model_results.png', dpi=300, bbox_inches='tight')
        print("Results saved to 'model_results.png'")
    plt.show()

# =========================================
# Main execution hoàn chỉnh
# =========================================
if __name__ == "__main__":
    print("=== Advanced LSTM Registration Prediction ===")
    
    try:
        # Load và split data
        (X_train, y_train, X_val, y_val, X_test, y_test,
         scaler, years, features_scaled, target_idx) = load_and_split_data('data.csv', seq_length=3)
        
        input_size = features_scaled.shape[1]
        num_features = input_size
        
        # Train model
        print("\n--- Training Model ---")
        model, train_losses, val_losses = train_model(
            X_train, y_train, X_val, y_val, 
            input_size=input_size,
            epochs=300,
            batch_size=min(32, len(X_train))  # Adaptive batch size
        )
        
        # Evaluate
        print("\n--- Evaluation ---")
        feature_cols = ['Registrations'] + [f'Diem_{i}' for i in range(10)]  # Adjust based on your data
        test_pred_orig, test_true_orig = evaluate_model(
            model, X_test, y_test, scaler, feature_cols, target_idx
        )
        
        # Future predictions
        print("\n--- Future Predictions ---")
        seq_length = 3
        last_seq = features_scaled[-seq_length:]
        future_years = list(range(years[-1] + 1, years[-1] + 4))  # Next 3 years
        future_preds = predict_future(
            model, scaler, last_seq, future_years, 
            feature_cols, target_idx, num_features
        )
        
        print("\nFuture Predictions:")
        for year, pred in zip(future_years, future_preds):
            print(f"Year {year}: {int(pred):,} registrations")
        
        # Historical registrations cho plot
        historical_registrations = scaler.inverse_transform(
            np.column_stack([features_scaled[:, target_idx], 
                           np.zeros((len(features_scaled), num_features-1))])
        )[:, 0]
        
        # Plot results
        print("\n--- Plotting Results ---")
        plot_results(train_losses, val_losses, years, historical_registrations,
                    future_years, future_preds, test_pred_orig, test_true_orig, save_fig=True)
        
        print("\n=== Training completed successfully! ===")
        print("Model saved as 'advanced_lstm_model.pth'")
        print("To load later: torch.load('advanced_lstm_model.pth')")
        
    except FileNotFoundError:
        print("Error: 'data.csv' not found!")
        print("Expected format:")
        print("Year,Registrations,Diem_Toan,Diem_Ly,Diem_Hoa,...")
        print("2015,800000,7.5,7.2,6.8,...")
        print("Create data.csv and try again.")
    except Exception as e:
        print(f"Error during execution: {str(e)}")
        print("Check your data format and try again.")